# 신경망 모델 구성하기

* 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(modeule)로 구성되어 있다.
* `torch.nn`: 신경망을 구성하는데 필요한 모든 구성 요소를 제공하는 네임스페이스
* `nn.Module`: pytorch의 모든 모듈의 상위 클래스
* 신경망은 다른 모듈(계층: layer)로 구성된 모듈이다. 이렇게 중첩한 구조로 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.
* FashionMNIST 데이터셋의 이미지를 분류하는 신경망을 구성해보자!

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1. 학습을 위한 장치 얻기
* 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습.
* `torch.cuda` 또는 `torch.backends.mps`가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_avaiable()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## 2. 클래스 정의하기
* 신경망 모델을 `nn.Module`의 하위클래스로 정의
* `__init__`에서 신경망 계층들을 초기화
* `nn.Module`을 상속받은 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산들을 구현

In [11]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__() # super(): 부모클래스의 임시적인 객체를 반환하여 부모클래스의 메소드를 사용할 수 있게 함
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x) #logit는 일반적으로 분류 모델에서 출력되는 값들을 나타내는 용어다.
    return logits

* `NeuralNetwork`의 인스턴스(instance)를 생성하고 이를 `device`로 이동한 뒤, 구조(structure)를 출력

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


* 모델을 사용하기 위해 입력 데이터 전달
* 이는 일부 백그라운드 연산들과 함께 모델의 `forward`를 실행한다.
* `model.forward()`를 직접 호출하면 안됨

* 모델에 입력을 전달하여 호출하면 2차원 텐서 반환
* 2차원 텐서의 dim = 0은 각 class에 대한 raw 예측값 10개, dim = 1에는 각 출력의 개별 값들이 해당
* 원시 예측값을 `nn.Softmax` 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

In [13]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


## 3. 모델 계층 (Layer)
* FashionMNIST 모델의 계층들을 살펴보자.
* 28 x 28 크기의 이미지 3개로 구성된 미니배치를 가져와 신경망을 통과할 때 어떤 일이 발생하는지 알아보자.

In [14]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
* `nn.Flatten`: 계층을 초기화하여 각 28 X 28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환 (dim = 0의 미니배치 차원은 유지)

In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
* 저장된 가중치(w) & 편향(b)를 사용해 선형 변환 (linear transformation)을 적용하는 선형 계층 모듈

In [22]:
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
* 선형 변환 이후에 적용되어 nonlinearity 도입 & 신경망이 다양한 현상을 학습할 수 있도록 함
* 이 모델에서는 `nn.ReLU`를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 다른 비선형성을 가진 다른 활성화를 도입할 수 있음

In [23]:
print(f"Before ReLU: {hidden1} \n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2764,  0.2738,  0.1709, -0.4190, -0.2762,  0.4262, -0.1862, -0.7226,
         -0.1122, -0.1257,  0.1998, -0.8544, -0.5842,  0.3435,  0.0999,  0.0392,
         -0.4012, -0.1264,  0.4220,  0.5195],
        [-0.4499,  0.3260,  0.1414, -0.5004, -0.3311,  0.2501, -0.1802, -0.5254,
         -0.4682, -0.5015, -0.0607, -0.4389, -0.5675,  0.5781,  0.1414,  0.0257,
         -0.0195, -0.0135,  0.4963,  0.3422],
        [-0.2331,  0.1193, -0.2282, -0.2070, -0.2654,  0.6217, -0.1826, -0.5232,
         -0.2234, -0.3016, -0.3195, -0.5898, -0.3203,  0.4328,  0.0721, -0.1271,
         -0.3701, -0.3192,  0.6468,  0.4962]], grad_fn=<AddmmBackward0>) 


After ReLU: tensor([[0.0000, 0.2738, 0.1709, 0.0000, 0.0000, 0.4262, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1998, 0.0000, 0.0000, 0.3435, 0.0999, 0.0392, 0.0000, 0.0000,
         0.4220, 0.5195],
        [0.0000, 0.3260, 0.1414, 0.0000, 0.0000, 0.2501, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.5781, 0.1

### nn.Sequential
* `nn.Sequential`: 순서를 갖는 모듈의 컨테이너
* 데이터는 정의된 순서로 모든 모듈들을 통해 전달
* 순차 컨테이너(sequential container)를 사용해 아래의 `seq_modules`와 같은 신경망을 빠르게 만들 수 있음


In [24]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax
* 신경망의 마지막 선형 계층은 `nn.Softmax`모듈에 전달될 ([-infty, infty]) 범위의 raw value인 `logits`를 반환
* `logits`: 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정(scale)
* `dim` parameter: 값의 합이 1이 되는 차원을 나타냄


In [25]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

### 모델 매개변수
* 신경망 내부의 많은 계층들은 매개변수화(parameterize)된다. -> 학습 중에 최적화되는 가중치와 편향과 이어짐
* `nn.Module`을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track) & 모델의 `parameters()` 및 `named_parameters()` 메소드로 모든 매개변수에 접근 가능
"

In [ ]:
print(f)